In [164]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
#import pyLDAvis.gensim_models
#from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [182]:
full_cv_df = pd.read_csv(r'C:\Users\Cohan\Documents\שבתון צחי\Resumes\DB4\Resume.csv')
cv_df = full_cv_df.copy().iloc[0:200]

In [143]:
nlp = spacy.load('en_core_web_lg')

In [165]:
skills = r'C:\Users\Cohan\Documents\שבתון צחי\Resumes\jz_skill_patterns.jsonl'

In [166]:
ruler = nlp.add_pipe('entity_ruler',before='ner')

ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'entity_ruler', 'ner']

In [168]:
ruler.from_disk(skills)

In [169]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset

def unique_skills(x):
    return list(set(x))

In [183]:
clean_cv_df = []
for i in range(cv_df.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        cv_df["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean_cv_df.append(str(review))

In [184]:
cv_df["Clean_Resume"] = clean_cv_df
cv_df["skills"] = cv_df["Clean_Resume"].str.lower().apply(get_skills)
cv_df["skills"] = cv_df["skills"].apply(unique_skills)
cv_df.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr administrator marketing associate hr admini...,"[support, swift, advertising, business, accoun..."
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist u hr operation summary versatile...,"[support, material, project management, advert..."
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr director summary 20 year experience recruit...,"[monitoring, support, tracking system, project..."
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist summary dedicated driven dynamic...,"[process management, monitoring, database, cer..."
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr manager skill highlight hr skill hr departm...,"[support, project management, box, business, d..."


In [186]:
my_cv = nlp(cv_df["Resume_str"].iloc[1])
displacy.render(my_cv, style="ent", jupyter=True)